In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pymc3 as pm
import pandas as pd
import os

np.random.seed(123)

%matplotlib inline
plt.style.use('ggplot')

import matplotlib

text_size = 20

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['axes.titlesize'] = text_size
matplotlib.rcParams['axes.labelsize'] = text_size - 2
matplotlib.rcParams['xtick.labelsize'] = text_size - 4
matplotlib.rcParams['ytick.labelsize'] = text_size - 4

# UWAGI:
* `MaskedArray` z `numpy` pozawala wepchnac brakujace wartosci (podobnie `pandas`owy `DataFrame` z wartosciami `NaN`);

In [ ]:
from datetime import datetime
DF_reddb = pd.read_csv('reddb.csv')

# przeprocesuj kolumne `Hour()`
DF_reddb['hour'] = DF_reddb['Hour()'].apply(lambda s: datetime.strptime(s, '%H %d/%m/%Y'))
DF_reddb['day'] = DF_reddb['Hour()'].apply(lambda s: datetime.strptime(s, '%H %d/%m/%Y'))

In [ ]:
# wykresy statow

stats = ['imp', 'index', 'pro_index', 'pro_scroll_8_8', 'con1']

fig = plt.figure(figsize = (30, 5))

for stat_number, stat_name in enumerate(stats):
    ax = fig.add_subplot(1, len(stats), stat_number + 1)
    ax.hist(DF_reddb[stat_name])
    ax.set_xlabel('log10({0})'.format(stat_name))
    if stat_number == 0:
        ax.set_ylabel('log10(count)')
    ax.set_xscale('log')
    ax.set_yscale('log')

In [ ]:
DF_reddb.show()

# Pierwsza proba stworzenia modelu

$$ \sigma \sim Exponential(50) $$

$$ \nu \sim Exponential(.1) $$

$$ s_i \sim Normal(s_{i-1}, \sigma^{-2}) $$

$$ log(\frac{y_i}{y_{i-1}}) \sim t(\nu, 0, exp(-2 s_i)) $$

In [1]:
import numpy as np
import pymc3 as pm
from pymc3.distributions.timeseries import GaussianRandomWalk

N = 10**3
succ = np.array([12, 12, 13, 12, 9, 7, 2, 0])
tries = np.array([1.0, 1.1, 1.5, 1.6, 1.1, 0.9, 0.9, 0.2]) * N
n = len(succ)




/usr/local/lib/python3.4/dist-packages/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [ ]:
N = 10**3
kids = np.array([1.0, 1.1, 1.5, 1.6, 1.1, 0.9, 0.9, 0.2]) * N

rsv_cases = np.array([12, 12, 13, 12, 9, 7, 2, 0])

n = len(rsv_cases)

with pm.Model() as test_model:
    
    # Prior probability
    prop = pm.Beta('prev_rsv', 1, N, shape = n)
    
    # Number of 1 y.o. in Amman
    n_amman = pm.Binomial('n_amman', kids, prop, shape = n)
    
    # Likelihood for number with RSV in hospital (assumes Pr(hosp | RSV) = 1)
    y_hosp = pm.Binomial('y_hosp', y_amman, market_share, observed = rsv_cases)

    trace = pm.sample(20000, step = pm.Metropolis())

In [5]:
with pm.Model() as test_model:

    trace = pm.sample(10000, step=pm.Metropolis())

    step1 = pm.NUTS(scaling=trace[-1], vars=[prev_rsv])
    step2 = pm.Metropolis(vars=[market_share, n_amman, y_amman], tune_interval=10)

    trace = pm.sample(5000, step=(step1, step2), start=trace[-1])

Help on class Binomial in module pymc3.distributions.discrete:

class Binomial(pymc3.distributions.distribution.Discrete)
 |  Binomial log-likelihood.
 |  
 |  The discrete probability distribution of the number of successes
 |  in a sequence of n independent yes/no experiments, each of which
 |  yields success with probability p.
 |  
 |  .. math:: f(x \mid n, p) = \binom{n}{x} p^x (1-p)^{n-x}
 |  
 |  ========  ==========================================
 |  Support   :math:`x \in \{0, 1, \ldots, n\}`
 |  Mean      :math:`n p`
 |  Variance  :math:`n p (1 - p)`
 |  ========  ==========================================
 |  
 |  Parameters
 |  ----------
 |  n : int
 |      Number of Bernoulli trials (n >= 0).
 |  p : float
 |      Probability of success in each trial (0 < p < 1).
 |  
 |  Method resolution order:
 |      Binomial
 |      pymc3.distributions.distribution.Discrete
 |      pymc3.distributions.distribution.Distribution
 |      builtins.object
 |  
 |  Methods defined here:
 